In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Homework 3

## numphy re-implementation

Read the code below and rewrite it using only numpy functions (without for loop,
if-else statements). Benchmark your implementation. How much is it faster than
the naive python implementation? (hint: use np.isin)

```python
def func1(names, db):
    idx = []
    for i in names:
        if i in db:
            idx.append(db.index(i))
    return idx
```

In [25]:
def func1(names, db):
    idx = []
    for i in names:
        if i in db:
            idx.append(db.index(i))
    return idx

In [82]:
def np_func1(names, db):
    db_index_array = np.arange(0, len(db))
    mask_isin = np.isin(db, names)
    return db_index_array[mask_isin]

In [89]:
np.random.seed(10)
names = np.random.randint(1,1000000,1000)        # List of names
db = list(np.random.randint(1,1000000,100000))   # List of db
print("func1(names, db) = \n", func1(names, db), "\n")
print("np_func1(names, db) = \n", np_func1(names, db), "\n")

func1(names, db) = 
 [97358, 42056, 28612, 5485, 66173, 59898, 42474, 77990, 65178, 70723, 27672, 13898, 38698, 18721, 9703, 63752, 20848, 59296, 39972, 41716, 81619, 47959, 41534, 91527, 23610, 98266, 97935, 58667, 93717, 73187, 74537, 14401, 12998, 50766, 83263, 67754, 20345, 82561, 55279, 86879, 63449, 75845, 59755, 61268, 24626, 861, 72014, 8077, 5170, 86338, 57254, 48421, 20526, 60184, 8027, 52049, 35508, 5349, 82279, 1466, 66507, 1309, 81715, 20149, 9138, 22969, 17412, 31165, 74871, 46622, 35903, 12537, 78670, 11756, 29497, 77553, 2225, 57547, 50210, 50987, 37178, 53309, 98801, 73966, 53085, 45473, 12641, 42578, 5378, 13377, 50055, 58262, 61445, 83996, 17195, 76289, 17365, 65887, 34917, 55814, 23103, 68168, 63920, 19811, 84982, 74199] 

np_func1(names, db) = 
 [  861  1309  1466  2225  5170  5349  5378  5485  8027  8077  9138  9703
 11756 12537 12641 12998 13377 13898 14401 17195 17365 17412 18721 19811
 20149 20210 20345 20526 20848 22969 23103 23610 24626 27672 28612 29497
 311

In [87]:
%timeit func1(names, db)
%timeit np_func1(names, db)  # About 100 times faster

2.5 s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
27 ms ± 785 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## largest 3 distances, using numpy 

Use only numpy functions (without for loop, if-else statements) to find the
largest 3 distances between points if either points is inside the circle of
raidus r0. Then return these points. E.g.

```python
>>> coordinates = np.array(
[[0.968, 0.313],
 [0.692, 0.876],
 [0.895, 0.085],
 [0.039, 0.17 ],
 [0.878, 0.098]]
)
>>> cirle_r0 = 0.9
>>> print(search_pairs(coordinates, cirle_r0))

# The indices of the relevant points in the original coordinates
[[2 1]
 [3 0]
 [3 1]]
```

In [119]:
# Given a set of points, find the largest 3 distances between points if either point is inside the circle with raidus r0
def search_pairs(coordinates, r0):

    pair_dists = np.linalg.norm(coordinates[:, np.newaxis, :] - coordinates, axis=2)  # pairwise distance between points
    points_in_r0 = np.linalg.norm(coordinates, axis=1) < r0     # mask, True if the point is inside the circle
    mask_r0 = points_in_r0[:, np.newaxis] ^ points_in_r0        # mask, True if either point of (pt1, pt2) is inside the circle
    mask_uniq_dists = np.tril(np.ones_like(mask_r0), k=-1)      # lower triangular 1's of mask_r0, to drop duplicates
    uniq_dists = pair_dists[mask_r0 & mask_uniq_dists]          # valid distances
    uniq_dists_top3 = np.partition(uniq_dists, -3)[-3:]         # largest 3 valid distances
    uniq_indices = np.argwhere(mask_r0 & mask_uniq_dists)       # indices of valid distances, using np.argwhere 
    return uniq_indices[np_func1(uniq_dists, uniq_dists_top3)]  # indices of the relevant points in the original coordinates

In [120]:
coordinates = np.array([[0.968, 0.313],
                        [0.692, 0.876],
                        [0.895, 0.085],
                        [0.039, 0.17 ],
                        [0.878, 0.098]])
cirle_r0 = 0.9
print(search_pairs(coordinates, cirle_r0))

[[2 0]
 [2 1]
 [3 0]]
